<a href="https://colab.research.google.com/github/NousForFreedom/taxi_call_service_pubsub/blob/main/taxi_plaform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install solace-pubsubplus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 10.7 MB/s eta 0:00:00


In [ ]:
import os
import platform
import time
import calendar
import math

# Import Solace Python  API modules from the solace package
from solace.messaging.messaging_service import MessagingService, ReconnectionListener, ReconnectionAttemptListener, \
            ServiceInterruptionListener, RetryStrategy, ServiceEvent
from solace.messaging.resources.topic import Topic
from solace.messaging.errors.pubsubplus_client_error import PubSubPlusClientError
from solace.messaging.publisher.request_reply_message_publisher import RequestReplyMessagePublisher

# Import Solace Python  API modules
from solace.messaging.messaging_service import MessagingService, ReconnectionListener, ReconnectionAttemptListener, ServiceInterruptionListener, RetryStrategy, ServiceEvent
from solace.messaging.errors.pubsubplus_client_error import PubSubPlusClientError
from solace.messaging.publisher.direct_message_publisher import PublishFailureListener, FailedPublishEvent
from solace.messaging.resources.topic_subscription import TopicSubscription
from solace.messaging.receiver.message_receiver import MessageHandler
from solace.messaging.config.solace_properties.message_properties import APPLICATION_MESSAGE_ID
from solace.messaging.resources.topic import Topic
from solace.messaging.receiver.inbound_message import InboundMessage

# Import Solace Python  API modules from the solace package
from solace.messaging.messaging_service import MessagingService, ReconnectionListener, ReconnectionAttemptListener, \
            ServiceInterruptionListener, RetryStrategy, ServiceEvent
from solace.messaging.resources.topic import Topic
from solace.messaging.resources.topic_subscription import TopicSubscription
from solace.messaging.receiver.request_reply_message_receiver import RequestMessageHandler, InboundMessage, \
            RequestReplyMessageReceiver, Replier


In [ ]:
# Broker Config
broker_props = {
  "solace.messaging.transport.host": "tcp://mr-connection-hcnx0u899eb.messaging.solace.cloud:55555",
  "solace.messaging.service.vpn-name": "aiot",
  "solace.messaging.authentication.scheme.basic.username": "Hanbinheo",
  "solace.messaging.authentication.scheme.basic.password": "ajou",
}

## Ride request **receive**

In [ ]:
import time
import calendar
import platform
import os

if platform.uname().system == 'Windows':
    os.environ["PYTHONUNBUFFERED"] = "1"  # Disable stdout buffer

TOPIC_PREFIX = "solace/taxi/python"

# Handle received messages
class RequestMessageHandlerImpl(RequestMessageHandler):
    def __init__(self, message_builder):
        self.message_builder = message_builder
        self.responses = []  # To store the responses
        self.extracted_data = []  # To store the extracted values

    def on_message(self, request: InboundMessage, replier: Replier):
        # Check if the payload is a String or Byte, decode if its the later
        payload = request.get_payload_as_string() if request.get_payload_as_string() != None else request.get_payload_as_bytes()
        if isinstance(payload, bytearray):
            print(f"Received a message of type: {type(payload)}. Decoding to string")
            payload = payload.decode()

        print(f'<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
        print(f'Received request payload:' + payload)
        print(f'----------------------------')

        # Extract information from the payload
        try:
            details = {}
            for part in payload.split(', '):
                key, value = part.split(': ')
                details[key.strip()] = value.strip()

            timestamp = details.get("Timestamp")
            user_id = details.get("USER-ID")
            start_location = details.get("StartLocation")
            destination = details.get("Destination")

            # Store the extracted values in a dictionary
            extracted_values = {
                "Timestamp": timestamp,
                "USER-ID": user_id,
                "StartLocation": start_location,
                "Destination": destination
            }
            self.extracted_data.append(extracted_values)

            print(f"Extracted Values:\nTimestamp: {timestamp}\nUSER-ID: {user_id}\nStartLocation: {start_location}\nDestination: {destination}")
        except Exception as e:
            print(f"Failed to parse payload: {payload} with error: {e}")
            response = 'Invalid payload format'
        else:
            response = f"Received request from {user_id} at {timestamp} moving from {start_location} to {destination}"

        self.responses.append(response)  # Store the response

        message_id = request.get_application_message_id()
        if replier is not None:
            outbound_msg = self.message_builder \
                            .with_application_message_id(f'{message_id}')\
                            .build(response)
            replier.reply(outbound_msg)
            print(f'>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
            print(f'Replied with response payload: {outbound_msg.get_payload_as_string()}')
            print(f'----------------------------')
        else:
            print(f'Invalid request, reply_to not set')

# Inner classes for error handling
class ServiceEventHandler(ReconnectionListener, ReconnectionAttemptListener, ServiceInterruptionListener):
    def on_reconnected(self, e: ServiceEvent):
        print("\non_reconnected")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_reconnecting(self, e: "ServiceEvent"):
        print("\non_reconnecting")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_service_interrupted(self, e: "ServiceEvent"):
        print("\non_service_interrupted")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

request_topic = TOPIC_PREFIX + '/direct/ride_request/>'
print(f'\nSubscribing to topic {request_topic}')

# Build A messaging service with a reconnection strategy of 20 retries over an interval of 3 seconds
# Note: The reconnections strategy could also be configured using the broker properties object
messaging_service = MessagingService.builder().from_properties(broker_props)\
                    .with_reconnection_retry_strategy(RetryStrategy.parametrized_retry(20,3))\
                    .build()

# Blocking connect thread
messaging_service.connect()
print(f'\nMessaging Service connected? {messaging_service.is_connected}')

# Event Handling for the messaging service
service_handler = ServiceEventHandler()
messaging_service.add_reconnection_listener(service_handler)
messaging_service.add_reconnection_attempt_listener(service_handler)
messaging_service.add_service_interruption_listener(service_handler)

# Create a direct message replier
direct_replier: RequestReplyMessageReceiver = messaging_service.request_reply() \
                                                .create_request_reply_message_receiver_builder() \
                                                .build(TopicSubscription.of(request_topic))

# Blocking Start thread
direct_replier.start()

# Prepare outbound message payload and body
outbound_msg_builder = messaging_service.message_builder() \
                .with_property("application", "taxi") \
                .with_property("language", "Python")

print("\nSend a KeyboardInterrupt to stop receiving\n")

try:
    # Callback for received messages
    request_handler = RequestMessageHandlerImpl(outbound_msg_builder)
    direct_replier.receive_async(request_handler)
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print('\nDisconnecting Messaging Service')
finally:
    print('\nTerminating receiver')
    direct_replier.terminate()
    print('\nDisconnecting Messaging Service')
    messaging_service.disconnect()

    # Print all stored responses
    print("\nStored responses:")
    for response in request_handler.responses:
        print(response)



Subscribing to topic solace/taxi/python/direct/ride_request/>

Messaging Service connected? True

Send a KeyboardInterrupt to stop receiving

<<<<<<<<<<<<<<<<<<<<<<<<<<<<
Received request payload:
Timestamp: 1719106276, USER-ID: USER12345, StartLocation: 41.40338.2.17403, Destination: 123.52341.76.45786
----------------------------
Extracted Values:
Timestamp: 1719106276
USER-ID: USER12345
StartLocation: 41.40338.2.17403
Destination: 123.52341.76.45786
>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Replied with response payload: Received request from USER12345 at 1719106276 moving from 41.40338.2.17403 to 123.52341.76.45786
----------------------------

Disconnecting Messaging Service

Terminating receiver

Disconnecting Messaging Service

Stored responses:
Received request from USER12345 at 1719106276 moving from 41.40338.2.17403 to 123.52341.76.45786


In [ ]:
user_id = request_handler.extracted_data[0]['USER-ID']
for data in request_handler.extracted_data:
        print(data)

{'Timestamp': '1719106276', 'USER-ID': 'USER12345', 'StartLocation': '41.40338.2.17403', 'Destination': '123.52341.76.45786'}


In [ ]:
request_handler.extracted_data

[{'Timestamp': '1719106276',
  'USER-ID': 'USER12345',
  'StartLocation': '41.40338.2.17403',
  'Destination': '123.52341.76.45786'}]

### **PickRequest**

In [ ]:
# Import Solace Python  API modules from the solace package
from solace.messaging.messaging_service import MessagingService, ReconnectionListener, ReconnectionAttemptListener, \
                                                ServiceInterruptionListener, RetryStrategy, ServiceEvent
from solace.messaging.resources.topic import Topic
from solace.messaging.publisher.direct_message_publisher import PublishFailureListener, FailedPublishEvent
from solace.messaging.resources.topic_subscription import TopicSubscription
from solace.messaging.receiver.message_receiver import MessageHandler, InboundMessage
from solace.messaging.errors.pubsubplus_client_error import PubSubPlusClientError

In [ ]:
# Constants

MSG_COUNT = 5  # Limit of messages to be sent for demonstration


# Inner classes for error handling
class ServiceEventHandler(ReconnectionListener, ReconnectionAttemptListener, ServiceInterruptionListener):
    def on_reconnected(self, e: ServiceEvent):
        print("\non_reconnected")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_reconnecting(self, e: "ServiceEvent"):
        print("\non_reconnecting")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_service_interrupted(self, e: "ServiceEvent"):
        print("\non_service_interrupted")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

class PublisherErrorHandling(PublishFailureListener):
    def on_failed_publish(self, e: "FailedPublishEvent"):
        print("on_failed_publish")

# Build and connect the messaging service
messaging_service = MessagingService.builder().from_properties(broker_props)\
                    .with_reconnection_retry_strategy(RetryStrategy.parametrized_retry(20, 3))\
                    .build()
messaging_service.connect()
print(f'Messaging Service connected? {messaging_service.is_connected}')

# Event Handling for the messaging service
service_handler = ServiceEventHandler()
messaging_service.add_reconnection_listener(service_handler)
messaging_service.add_reconnection_attempt_listener(service_handler)
messaging_service.add_service_interruption_listener(service_handler)

# Create a direct message publisher and start it
direct_publisher = messaging_service.create_direct_message_publisher_builder().build()
direct_publisher.set_publish_failure_listener(PublisherErrorHandling())
direct_publisher.start()
print(f'Direct Publisher ready? {direct_publisher.is_ready()}')

print("\nSend a KeyboardInterrupt to stop publishing\n")

try:
    count = 1
    while True:
        for data in request_handler.extracted_data:

            if count > MSG_COUNT:
                break

            # Extract data from the current dictionary in the list
            timestamp = str(calendar.timegm(time.gmtime()))
            start_location = data.get("StartLocation")
            destination = data.get("Destination")
            #user-id

            # Build the outbound message with the extracted data
            outbound_msg = messaging_service.message_builder() \
                .with_application_message_id(f'PickUpRequest_{count}') \
                .with_property("Timestamp", timestamp) \
                .with_property("StartLocation", start_location) \
                .with_property("Destination", destination) \
                .build(f'PickUpRequest - {timestamp}: from {start_location} to {destination}')

            # Define the topic (e.g., could be based on driver proximity or some other logic)
            topic = Topic.of(TOPIC_PREFIX + f'/direct/PickRequest/driver_destination_A/{count}')
            direct_publisher.publish(destination=topic, message=outbound_msg)

            print(f'Published PickUpRequest message to {topic}')
            count += 1
            time.sleep(0.1)

        # Reset count after publishing up to MSG_COUNT messages
        print("\n")
        count = 1
        time.sleep(1)

except KeyboardInterrupt:
    print('\nTerminating Publisher')
    direct_publisher.terminate()
    print('\nDisconnecting Messaging Service')
    messaging_service.disconnect()


Messaging Service connected? True
Direct Publisher ready? True

Send a KeyboardInterrupt to stop publishing

Published PickUpRequest message to topic : solace/taxi/python/direct/PickRequest/driver_destination_A/1 


Published PickUpRequest message to topic : solace/taxi/python/direct/PickRequest/driver_destination_A/1 


Published PickUpRequest message to topic : solace/taxi/python/direct/PickRequest/driver_destination_A/1 


Published PickUpRequest message to topic : solace/taxi/python/direct/PickRequest/driver_destination_A/1 


Published PickUpRequest message to topic : solace/taxi/python/direct/PickRequest/driver_destination_A/1 


Published PickUpRequest message to topic : solace/taxi/python/direct/PickRequest/driver_destination_A/1 


Published PickUpRequest message to topic : solace/taxi/python/direct/PickRequest/driver_destination_A/1 


Published PickUpRequest message to topic : solace/taxi/python/direct/PickRequest/driver_destination_A/1 


Published PickUpRequest message to 

### **PickupRequestResponse reply**

In [ ]:
# Ensure stdout is unbuffered on Windows
if platform.uname().system == 'Windows':
    os.environ["PYTHONUNBUFFERED"] = "1"

# Constants
request_topic = f"{TOPIC_PREFIX}/direct/request/>"

# Inner class for connection error handling
class ServiceEventHandler(ReconnectionListener, ReconnectionAttemptListener, ServiceInterruptionListener):
    def on_reconnected(self, e: ServiceEvent):
        print("\non_reconnected")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_reconnecting(self, e: "ServiceEvent"):
        print("\non_reconnecting")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_service_interrupted(self, e: "ServiceEvent"):
        print("\non_service_interrupted")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

# Build and connect the messaging service
messaging_service = MessagingService.builder().from_properties(broker_props) \
                                     .with_reconnection_retry_strategy(RetryStrategy.parametrized_retry(20, 3)) \
                                     .build()
messaging_service.connect()
print(f'Messaging Service connected? {messaging_service.is_connected}')

# Event Handling for the messaging service
service_handler = ServiceEventHandler()
messaging_service.add_reconnection_listener(service_handler)
messaging_service.add_reconnection_attempt_listener(service_handler)
messaging_service.add_service_interruption_listener(service_handler)

# Create a direct message replier
direct_replier: RequestReplyMessageReceiver = messaging_service.request_reply() \
                                                .create_request_reply_message_receiver_builder() \
                                                .build(TopicSubscription.of(request_topic))
direct_replier.start()
print(f'Replier started on topic: {request_topic}')

# Handler for processing incoming requests
class PickupRequestHandler(RequestMessageHandler):
    def __init__(self, message_builder):
        self.message_builder = message_builder
        self.is_ride_assigned = False  # To track if the ride is already assigned. 선착순 판정
        self.first_response_sent = False  # To track if the first response has been sent

    def on_message(self, request: InboundMessage, replier: Replier):
        payload = request.get_payload_as_string()
        print(f'\n\nReceived request payload: {payload}')

        try:
            details = {part.split(': ')[0]: part.split(': ')[1] for part in payload.split(', ')}
            timestamp = str(calendar.timegm(time.gmtime()))
            ride_id = details.get("Ride-ID")
            driver_result = details.get("Result")  # Expected to be "True" indicating acceptance


            if not self.is_ride_assigned and driver_result == "True":
                # Assign the ride to the first driver who responds with True
                response_result = "True"
                self.is_ride_assigned = True
            else:
                # Reject subsequent drivers
                response_result = "False"


            response_message = f"Result: {response_result}"
            outbound_msg = self.message_builder.with_application_message_id(request.get_application_message_id()) \
                                               .build(response_message)
            replier.reply(outbound_msg)
            print(f'Replied with: {response_message}')

            if response_result == "True":
                self.first_response_sent = True
                print("Ride has been assigned to the first driver.")
                # Here you might want to include logic to stop listening to further requests or take other actions

        except Exception as e:
            print(f'Failed to parse payload: {payload} with error: {e}')

# Prepare outbound message payload and body
outbound_msg_builder = messaging_service.message_builder() \
                .with_property("application", "samples") \
                .with_property("language", "Python")

# Register the handler
request_handler = PickupRequestHandler(outbound_msg_builder)
direct_replier.receive_async(request_handler)

print("Listening for pickup requests. Press Ctrl+C to exit.")
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\nExiting...")

finally:
    direct_replier.terminate()
    messaging_service.disconnect()
    print('Messaging service disconnected.')

Messaging Service connected? True
Replier started on topic: solace/taxi/python/direct/request/>
Listening for pickup requests. Press Ctrl+C to exit.


Received request payload: Timestamp: 1719106965, Ride-ID: RIDE12345, Result: True
Replied with: Result: True
Ride has been assigned to the first driver.

Exiting...
Messaging service disconnected.


### **RideRequestResponse**

In [ ]:

# Inner class for handling connection events
class ServiceEventHandler(ReconnectionListener, ReconnectionAttemptListener, ServiceInterruptionListener):
    def on_reconnected(self, e: ServiceEvent):
        print("\non_reconnected")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_reconnecting(self, e: "ServiceEvent"):
        print("\non_reconnecting")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_service_interrupted(self, e: "ServiceEvent"):
        print("\non_service_interrupted")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

# Build and connect the messaging service
messaging_service = MessagingService.builder().from_properties(broker_props) \
                                     .with_reconnection_retry_strategy(RetryStrategy.parametrized_retry(20, 3)) \
                                     .build()
messaging_service.connect()
print(f'Messaging Service connected? {messaging_service.is_connected}')

# Event Handling for the messaging service
service_handler = ServiceEventHandler()
messaging_service.add_reconnection_listener(service_handler)
messaging_service.add_reconnection_attempt_listener(service_handler)
messaging_service.add_service_interruption_listener(service_handler)

# Create a persistent message publisher for sending RideRequestResponse to the user
direct_publisher = messaging_service.create_direct_message_publisher_builder().build()
direct_publisher.start()

print(f'Persistent Publisher ready? {direct_publisher.is_ready()}')

# Define the topic to send RideRequestResponse message to the user
user_ride_topic = Topic.of(TOPIC_PREFIX + f'/guaranteed/ride_request_response/{user_id}')

# Build the message payload
timestamp = str(calendar.timegm(time.gmtime()))
eta = "224"  # Example estimated time of arrival
taxi_number = "24가5735"  # Example taxi number

payload = f"Timestamp: {timestamp}, Estimated Time of Arrival: {eta}, TaxiNumber: {taxi_number}"

try:
    outbound_msg = messaging_service.message_builder() \
        .with_application_message_id(f'RideRequestResponse_{user_id}_{timestamp}') \
        .build(payload)

    direct_publisher.publish(destination=user_ride_topic, message=outbound_msg)
    print(f'Sent RideRequestResponse message to topic: {user_ride_topic}')
except Exception as e:
    print(f'Failed to send RideRequestResponse message: {e}')
finally:
    messaging_service.disconnect()

Messaging Service connected? True
Persistent Publisher ready? True
Sent RideRequestResponse message to topic: topic : solace/taxi/python/guaranteed/ride_request_response/USER12345 


### **PickupComplete 받기**

In [ ]:
# Constants

pickup_complete_topic = f"{TOPIC_PREFIX}/direct/pickup_complete/>"

# Shared data structure for tracking pickups
pickup_data = []

# Inner class for handling connection events
class ServiceEventHandler(ReconnectionListener, ReconnectionAttemptListener, ServiceInterruptionListener):
    def on_reconnected(self, e: ServiceEvent):
        print("\non_reconnected")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_reconnecting(self, e: "ServiceEvent"):
        print("\non_reconnecting")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_service_interrupted(self, e: "ServiceEvent"):
        print("\non_service_interrupted")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

# Build and connect the messaging service
messaging_service = MessagingService.builder().from_properties(broker_props) \
                                     .with_reconnection_retry_strategy(RetryStrategy.parametrized_retry(20, 3)) \
                                     .build()
messaging_service.connect()
print(f'Messaging Service connected? {messaging_service.is_connected}')

# Event Handling for the messaging service
service_handler = ServiceEventHandler()
messaging_service.add_reconnection_listener(service_handler)
messaging_service.add_reconnection_attempt_listener(service_handler)
messaging_service.add_service_interruption_listener(service_handler)

# Create a direct message receiver for PickupComplete messages
direct_receiver: RequestReplyMessageReceiver = messaging_service.request_reply() \
                                                .create_request_reply_message_receiver_builder() \
                                                .build(TopicSubscription.of(pickup_complete_topic))
direct_receiver.start()
print(f'Receiver started on topic: {pickup_complete_topic}')

# Handler for processing incoming PickupComplete messages
class PickupCompleteHandler(RequestMessageHandler):
    def __init__(self, shared_data):
        self.shared_data = shared_data

    def on_message(self, request: InboundMessage, replier: Replier):
        payload = request.get_payload_as_string()
        print(f'>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n')
        print(f'Received PickupComplete payload: {payload}')
        print(f'\n----------------------------')

        try:
            details = {part.split(': ')[0]: part.split(': ')[1] for part in payload.split(', ')}
            timestamp = str(calendar.timegm(time.gmtime()))
            ride_id = details.get("Ride-ID")
            location = details.get("Location")

            # Store the pickup complete data
            self.shared_data.append(details)

            #print(f'Pickup complete for Ride-ID: {ride_id} at {location} recorded at {timestamp}')

        except Exception as e:
            print('')
            #print(f'Failed to parse payload: {payload} with error: {e}')

# Register the handler
pickup_handler = PickupCompleteHandler(pickup_data)
direct_receiver.receive_async(pickup_handler)

print("Listening for PickupComplete messages. Press Ctrl+C to exit.")
try:
    while True:
        # Additional logic can be added here to handle delayed pickups or request status updates
        time.sleep(1)
except KeyboardInterrupt:
    print("Exiting...")

finally:
    direct_receiver.terminate()
    messaging_service.disconnect()
    print('Messaging service disconnected.')

Messaging Service connected? True
Receiver started on topic: solace/taxi/python/direct/pickup_complete/>
Listening for PickupComplete messages. Press Ctrl+C to exit.
>>>>>>>>>>>>>>>>>>>>>>>>>>>>

Received PickupComplete payload: PickupComplete - 1719107171: Ride RIDE12345 at 41.40338.2.17403

----------------------------
>>>>>>>>>>>>>>>>>>>>>>>>>>>>

Received PickupComplete payload: PickupComplete - 1719107334: Ride RIDE12345 at 41.40338.2.17403

----------------------------
Exiting...
Messaging service disconnected.


### **DropoffComplete** 받기

In [ ]:
# Constants
dropoff_complete_topic = f"{TOPIC_PREFIX}/direct/dropoff_complete/>"

# Shared data structure for tracking drop-offs
dropoff_data = []

# Inner class for handling connection events
class ServiceEventHandler(ReconnectionListener, ReconnectionAttemptListener, ServiceInterruptionListener):
    def on_reconnected(self, e: ServiceEvent):
        print("\non_reconnected")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_reconnecting(self, e: "ServiceEvent"):
        print("\non_reconnecting")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_service_interrupted(self, e: "ServiceEvent"):
        print("\non_service_interrupted")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

# Build and connect the messaging service
messaging_service = MessagingService.builder().from_properties(broker_props) \
                                     .with_reconnection_retry_strategy(RetryStrategy.parametrized_retry(20, 3)) \
                                     .build()
messaging_service.connect()
print(f'Messaging Service connected? {messaging_service.is_connected}')

# Event Handling for the messaging service
service_handler = ServiceEventHandler()
messaging_service.add_reconnection_listener(service_handler)
messaging_service.add_reconnection_attempt_listener(service_handler)
messaging_service.add_service_interruption_listener(service_handler)

# Create a direct message receiver for DropoffComplete messages
direct_receiver: RequestReplyMessageReceiver = messaging_service.request_reply() \
                                                .create_request_reply_message_receiver_builder() \
                                                .build(TopicSubscription.of(dropoff_complete_topic))
direct_receiver.start()
print(f'Receiver started on topic: {dropoff_complete_topic}')
ride_id = "RIDE12345"

# Handler for processing incoming DropoffComplete messages
class DropoffCompleteHandler(RequestMessageHandler):
    def __init__(self, shared_data):
        self.shared_data = shared_data

    def on_message(self, request: InboundMessage, replier: Replier):
        payload = request.get_payload_as_string()
        print(f'>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n')
        print(f'Received DropoffComplete payload: {payload}')
        print(f'\n----------------------------')

        try:
            details = {part.split(': ')[0]: part.split(': ')[1] for part in payload.split(', ')}
            timestamp = str(calendar.timegm(time.gmtime()))
            ride_id = details.get("Ride-ID")
            location = details.get("Location")

            # Store the drop-off complete data
            self.shared_data.append(details)

            print(f'Drop-off complete for Ride-ID: {ride_id} at {location} recorded at {timestamp}')

        except Exception as e:
            print(f'Failed to parse payload: {payload} with error: {e}')

# Register the handler
dropoff_handler = DropoffCompleteHandler(dropoff_data)
direct_receiver.receive_async(dropoff_handler)

print("Listening for DropoffComplete messages. Press Ctrl+C to exit.")
try:
    while True:
        # Additional logic can be added here to handle delayed drop-offs or request status updates
        time.sleep(1)
except KeyboardInterrupt:
    print("Exiting...")

finally:
    direct_receiver.terminate()
    messaging_service.disconnect()
    print('Messaging service disconnected.')

Messaging Service connected? True
Receiver started on topic: solace/taxi/python/direct/dropoff_complete/>
Listening for DropoffComplete messages. Press Ctrl+C to exit.
>>>>>>>>>>>>>>>>>>>>>>>>>>>>

Received DropoffComplete payload: DropoffComplete - 1719107354: Ride RIDE12345 at 123.52341.76.45786

----------------------------
Drop-off complete for Ride-ID: None at None recorded at 1719107354
Exiting...
Messaging service disconnected.


### **DropoffComplete**User



In [ ]:
service_complete = False

# Inner class for handling connection events
class ServiceEventHandler(ReconnectionListener, ReconnectionAttemptListener, ServiceInterruptionListener):
    def on_reconnected(self, e: ServiceEvent):
        print("\non_reconnected")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_reconnecting(self, e: "ServiceEvent"):
        print("\non_reconnecting")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_service_interrupted(self, e: "ServiceEvent"):
        print("\non_service_interrupted")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

# Build and connect the messaging service
messaging_service = MessagingService.builder().from_properties(broker_props) \
                                     .with_reconnection_retry_strategy(RetryStrategy.parametrized_retry(20, 3)) \
                                     .build()
messaging_service.connect()
print(f'Messaging Service connected? {messaging_service.is_connected}')

# Event Handling for the messaging service
service_handler = ServiceEventHandler()
messaging_service.add_reconnection_listener(service_handler)
messaging_service.add_reconnection_attempt_listener(service_handler)
messaging_service.add_service_interruption_listener(service_handler)

# Create a request-reply message receiver for DropoffCompleteUser requests
direct_requestor: RequestReplyMessagePublisher = messaging_service.request_reply() \
                                                                .create_request_reply_message_publisher_builder() \
                                                                .build()


direct_requestor.start()

# Define the topic to send requests to the user (driver)
request_topic = Topic.of(TOPIC_PREFIX + f'/direct/dropoff_complete_user_request/{user_id}')

# Handler for processing replies from the user (driver)
class DropoffCompleteUserReplyHandler(RequestMessageHandler):
    def on_message(self, request, replier):
        reply_payload = request.get_payload_as_string()
        print(f'Received DropoffCompleteUser reply: {reply_payload}')

        # Process the reply message
        try:
            details = {part.split(': ')[0]: part.split(': ')[1] for part in reply_payload.split(', ')}
            timestamp = details.get("Timestamp")
            result = details.get("Result")

            print(f'DropoffCompleteUser result: Timestamp {timestamp}, Result {result}')

        except Exception as e:
            print(f'Failed to parse payload: {reply_payload} with error: {e}')

# Prepare and send the request to the user (driver)
try:
    ask_message = messaging_service.message_builder() \
        .with_application_message_id(f'DropoffCompleteUserAsk_{user_id}') \
        .build("Did you arrive safely?")
    service_complete = True
    print(f'Sending DropoffCompleteUser request: Did you arrive safely?')
    publish_async = direct_requestor.publish(request_message=ask_message, \
                                                request_destination=request_topic,
                                                reply_timeout=3000)
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print(service_complete)
    print('\nTerminating Request-Reply Receiver')
    direct_receiver.terminate()
    messaging_service.disconnect()

Messaging Service connected? True
Sending DropoffCompleteUser request: Did you arrive safely?
True

Terminating Request-Reply Receiver
2024-06-23 01:50:50,156 [WARNING] solace.messaging.receiver: [_message_receiver.py:303]  [[SERVICE: 0x7c03b27622b0] [RECEIVER: 0x7c03b2770bb0]] Message receiver already terminated


In [ ]:
print(service_complete)

True


payment

In [ ]:
class ServiceEventHandler(ReconnectionListener, ReconnectionAttemptListener, ServiceInterruptionListener):
    def on_reconnected(self, e: ServiceEvent):
        print("\non_reconnected")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_reconnecting(self, e: "ServiceEvent"):
        print("\non_reconnecting")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_service_interrupted(self, e: "ServiceEvent"):
        print("\non_service_interrupted")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

# Build and connect the messaging service
messaging_service = MessagingService.builder().from_properties(broker_props) \
                                     .with_reconnection_retry_strategy(RetryStrategy.parametrized_retry(20, 3)) \
                                     .build()
messaging_service.connect()
print(f'Messaging Service connected? {messaging_service.is_connected}')

# Event Handling for the messaging service
service_handler = ServiceEventHandler()
messaging_service.add_reconnection_listener(service_handler)
messaging_service.add_reconnection_attempt_listener(service_handler)
messaging_service.add_service_interruption_listener(service_handler)

# Create a direct message publisher for sending PaymentStart message
direct_publisher = messaging_service.create_direct_message_publisher_builder().build()
direct_publisher.start()
print(f'Direct Publisher ready? {direct_publisher.is_ready()}')

# Define the topic to send PaymentStart message to the payment system
payment_start_topic = Topic.of(TOPIC_PREFIX + '/direct/payment/start')

# Build the message payload
payload = f"user_id: {user_id}, ride_id: {ride_id}"

if(service_complete) :
  try:
    outbound_msg = messaging_service.message_builder() \
        .with_application_message_id(f'PaymentStart_{user_id}_{ride_id}') \
        .build(payload)

    direct_publisher.publish(destination=payment_start_topic, message=outbound_msg)
    print(f'Sent PaymentStart message to topic: {payment_start_topic}')
  except Exception as e:
    print(f'Failed to send PaymentStart message: {e}')

  finally:
    print('\nDisconnecting Messaging Service')
    messaging_service.disconnect()



Messaging Service connected? True
Direct Publisher ready? True
Sent PaymentStart message to topic: topic : solace/taxi/python/direct/payment/start 

Disconnecting Messaging Service
